In [37]:
import requests
import re
import os
import pandas as pd
import folium
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import json

In [38]:
#zipcodes in LA
f = open('inputZips.txt', 'r')
allZips = [z.replace('\n','') for z in f.readlines()]
folder_name = '/raw'

In [39]:
def saveAndProcessResponse(idx, r):
    #parse out each store's info
    stores = re.findall(r'"storeNumber":.*?"slug"', r)
    file_name = str(idx) + ".txt"
    current_directory = os.path.realpath('.') + folder_name
    file_path = os.path.join(current_directory, file_name)
    f = open(file_path, "w+")
    f.write(stores)
    storeInfo = []
    for store in stores:
        #parse out info about each store
        storeInfo.append(list(re.findall(r'"storeNumber":"(.*?)".*?"name":"(.*?)".*?"latitude":(.*?),.*?"longitude":(.*?)}.*?"city":"(.*?)".*?"countrySubdivisionCode":"(.*?)".*?"postalCode":"(.*?)"', store)[0]))
    return storeInfo

In [40]:
# def saveAndProcessResponse(r):

In [41]:
def appendResponseData(idx,r):
    print(idx)

In [42]:
allStores = []
for idx,nonSplitLine in enumerate(allZips):
    line = nonSplitLine.split(",")
    if idx%10 == 0:
        print(idx,'/',len(allZips))
    #search for 100 stores centered in given zip
    # r = requests.get('https://www.starbucks.com/store-locator?map=34.216464,-118.656046,11z&place='+z)
    # 1923,Danvers,Massachusetts,MA,Essex,42.5694,-70.9425
    post = line[0]
    lat = line[1]
    lon = line[2]
    # urlll = 'https://www.starbucks.com/store-locator?map='+lat+','+lon+'z&place='+post
    urlll = 'https://www.starbucks.com/store-locator?map='+lat+','+lon+',11z'
    r = requests.get(urlll)
    if r.status_code != 200:
        continue
        # raise SystemExit
    storeInfoList = saveAndProcessResponse(idx, r.text)
    # appendResponseData(storeInfoList)
    # allStores += storeInfoList
    # storeInfoList = processResponse(r.text)
    # for storeInfo in storeInfoList:
    #     storeInfo[6] = storeInfo[6][:5]
    # allStores += storeInfoList

0 / 1
/Users/user/Documents/xcode/StarbucksStoreScraping/raw
/raw


FileNotFoundError: [Errno 2] No such file or directory: '/raw/0.txt'

In [ ]:
#construct non-duplicated list of stores
allStores = []
seenStoreIds = []
laStores = []
for store in allStores:
    if store[0] in seenStoreIds:
        continue
    else:
        laStores.append(store)
        seenStoreIds.append(store[0])

In [ ]:
#define columns of csv
dfSbux = pd.DataFrame(columns=['id', 'strLocation', 'latitude', 'longitude', 'city', 'state', 'zip'])

#store data
for i,col in enumerate(dfSbux.columns):
    dfSbux[col] = [item[i] for item in laStores]

#cast the latitude and longitude as floats
dfSbux.latitude = dfSbux.latitude.apply(lambda x: float(x))
dfSbux.longitude = dfSbux.longitude.apply(lambda x: float(x))

#save to csv
dfSbux.to_csv('starbucksInUS.csv', index=False)

In [ ]:
# dfSbux = pd.DataFrame(columns=['id', 'strLocation', 'latitude', 'longitude', 'city', 'state', 'zip'])
# for i,col in enumerate(dfSbux.columns):
#     dfSbux[col] = [item[i] for item in keepLAStores]

In [ ]:
# dfSbux.latitude = dfSbux.latitude.apply(lambda x: float(x))
# dfSbux.longitude = dfSbux.longitude.apply(lambda x: float(x))

In [ ]:
# laMap = folium.Map(location=[34.0522,-118.2437], tiles='Stamen Toner', zoom_start=10)
# for i,row in dfSbux.iterrows():
#     folium.CircleMarker((row.latitude,row.longitude), radius=3, weight=2, fill_color='red', fill_opacity=.9).add_to(laMap)

In [ ]:
# folium.GeoJson(laArea).add_to(laMap)